In [1]:
%load_ext autoreload
%autoreload 2
import os#
import torch
import argparse
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from models.networks import UnetGenerator, InceptionDown, PartialConv, InceptionUp
import numpy as np
from models.modules.shift_unet import InceptionUnetGeneratorShiftTriple
from util.util import wrapper_gmask

In [2]:
from options.train_options import TrainOptions 

In [3]:
dataroot = '/mnt/hdd2/AIM/DAGM/Class4_def/' # ENTER HERE THE PATH YOU WANT TO USE AS DATAROOT
options = '--dataroot {}'.format(dataroot).split(' ')

In [4]:
def get_parser(options=None):
    parser = TrainOptions()
    parser.parse(options=options)
    return parser

In [5]:
parser = get_parser(options=options)

----------------- Options ---------------
                batchSize: 1                             
                    beta1: 0.5                           
               bottleneck: 512                           
          checkpoints_dir: /mnt/hdd2/AIM/checkpoints/    
                constrain: MSE                           
           continue_train: False                         
                 dataroot: /mnt/hdd2/AIM/DAGM/Class4_def/	[default: /mnt/hdd2/AIM/DAGM/Class4]
             dataset_mode: aligned_resized               
             display_freq: 10                            
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
display_single_pane_ncols: 0                             
          display_winsize: 256                           
              epoch_count: 1                       

In [6]:
opt = parser.opt

In [7]:
mask = wrapper_gmask(opt)

/home/tchaton/virtualenvs/labelbox/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [8]:
mask.float().cuda()

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:3')

In [9]:
input_nc = 3
output_nc = 3
num_downs = 7
innerCos_list = []
shift_list = []
mask_global = mask

In [10]:
inception_unet = InceptionUnetGeneratorShiftTriple(input_nc, output_nc, num_downs, opt, innerCos_list, shift_list, mask_global.cuda())
inception_unet.cuda()

InceptionUnetGeneratorShiftTriple(
  (model): InceptionUnetSkipConnectionBlock(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): InceptionUnetSkipConnectionBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace)
          (1): InceptionDown(
            (act): LeakyReLU(negative_slope=0.2, inplace)
            (conv0_1x1_0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (bconv0_1x1_0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv0_1x1_1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (bconv0_1x1_1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (max_pool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv0_1x1_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (bconv0_1x1_2): BatchNorm2d(32, eps=1e-05, m

In [11]:
def count_params(model):
    nb_params = 0
    for param in model.parameters():
        if param is not None:
            nb_params += np.product(param.shape)
    return nb_params

In [12]:
count_params(inception_unet)

3541107

In [13]:
x_np = np.random.normal(0, 1, (1, 3, 256, 256))
x_tr = torch.FloatTensor(x_np)
inception_unet = inception_unet.cuda()

In [14]:
%time output = inception_unet(x_tr.cuda())

CPU times: user 120 ms, sys: 16 ms, total: 136 ms
Wall time: 135 ms


/home/tchaton/virtualenvs/labelbox/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [15]:
inception = InceptionDown(128, 256)

In [16]:
x_np = np.random.normal(0, 1, (1, 128, 64, 64))
x_tr = torch.FloatTensor(x_np)

In [17]:
%time output = inception(x_tr)

CPU times: user 1.14 s, sys: 72 ms, total: 1.21 s
Wall time: 84.4 ms


In [18]:
inception = InceptionUp(128, 32)

In [19]:
%timeit output = inception(x_tr)

36.5 ms ± 4.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
output.shape

torch.Size([1, 256, 32, 32])